Лабораторная работа №4.
Цель работы: познакомиться с библиотекой Optuna. Оптимизировать алгоритм градиентого спуска.

In [1]:
import numpy as np
import optuna
import sklearn.datasets
import sklearn.linear_model
import sklearn.metrics
import sklearn.model_selection
from sklearn.model_selection import train_test_split
import logging
import lightgbm as lgb
import numpy as np
import plotly

import sys
#Графики
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

Запишем функции

In [2]:
def func1(x, y):      # Функция Матьяса  -10:10
    return 0.26 * (x*x + y*y) - 0.48*x*y

def objective_f1(trial):
    x = trial.suggest_float('x', -10, 10)
    y = trial.suggest_float('y', -10, 10)
    return func1(x, y)

In [3]:
def func2(x, y): # Функция Химмельблау -5:5 
    return (x * x + y - 11)**2 + (x + y * y - 7)**2

def objective_f2(trial):
    x = trial.suggest_float('x', -5, 5)
    y = trial.suggest_float('y', -5, 5)
    return func2(x, y)


Запустим оптисизацию для первой функции

In [4]:
study_f1 = optuna.create_study()
study_f1.optimize(objective_f1, n_trials=100)

[I 2022-12-20 19:57:17,616] A new study created in memory with name: no-name-3c5d7d66-3ae2-4e45-b5b0-5fa92d605ebf
[I 2022-12-20 19:57:17,625] Trial 0 finished with value: 3.756418303689614 and parameters: {'x': -8.147331773396713, 'y': -5.369246522423046}. Best is trial 0 with value: 3.756418303689614.
[I 2022-12-20 19:57:17,633] Trial 1 finished with value: 0.9033021938943353 and parameters: {'x': -4.841431927050128, 'y': -4.38617081019149}. Best is trial 1 with value: 0.9033021938943353.
[I 2022-12-20 19:57:17,639] Trial 2 finished with value: 20.010510292161417 and parameters: {'x': -5.853135167305434, 'y': 3.0762317455328763}. Best is trial 1 with value: 0.9033021938943353.
[I 2022-12-20 19:57:17,647] Trial 3 finished with value: 15.610405114087936 and parameters: {'x': 6.759604030919267, 'y': -1.05973613926513}. Best is trial 1 with value: 0.9033021938943353.
[I 2022-12-20 19:57:17,653] Trial 4 finished with value: 29.495672478254207 and parameters: {'x': -8.94753054680232, 'y': 1

Выведем лучшие параметры

In [5]:
best_params_f1 = study_f1.best_params
found_x = best_params_f1["x"]
found_y = best_params_f1["y"]
print("Функция Матиаса:")
print(f"x: {found_x}, y: {found_y}, E: {func1(found_x, found_y)}")

Функция Матиаса:
x: 0.28202825794446484, y: 0.2762500202874753, E: 0.003125093367061363


Повторим для второй функции

In [6]:
study_f2 = optuna.create_study()
study_f2.optimize(objective_f2, n_trials=100)

[I 2022-12-20 19:57:20,204] A new study created in memory with name: no-name-97a75a2f-a86d-4992-b914-f60522754ff6
[I 2022-12-20 19:57:20,214] Trial 0 finished with value: 211.75787082677527 and parameters: {'x': 0.04560603659629692, 'y': 4.467514806813901}. Best is trial 0 with value: 211.75787082677527.
[I 2022-12-20 19:57:20,219] Trial 1 finished with value: 171.72393492620174 and parameters: {'x': 3.7027017704753895, 'y': 3.828070916358378}. Best is trial 1 with value: 171.72393492620174.
[I 2022-12-20 19:57:20,224] Trial 2 finished with value: 81.04469121543414 and parameters: {'x': -2.977621674898362, 'y': 1.0218009320371948}. Best is trial 2 with value: 81.04469121543414.
[I 2022-12-20 19:57:20,231] Trial 3 finished with value: 176.3547887842254 and parameters: {'x': -0.08862152169539339, 'y': -0.2903216891761069}. Best is trial 2 with value: 81.04469121543414.
[I 2022-12-20 19:57:20,237] Trial 4 finished with value: 233.18973358020162 and parameters: {'x': -0.029776945902060703,

In [7]:
best_params_f2 = study_f2.best_params
found_x = best_params_f2["x"]
found_y = best_params_f2["y"]
print("Функция Химмельблау:")
print(f"x: {found_x}, y: {found_y}, E: {func2(found_x, found_y)}")

Функция Химмельблау:
x: 3.0486464830057693, y: 1.8350783160594417, E: 0.3575950373837901


П

In [8]:
def objective(trial):
    data, target = sklearn.datasets.load_wine(return_X_y=True)
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    dtrain = lgb.Dataset(train_x, label=train_y)
    dvalid = lgb.Dataset(valid_x, label=valid_y)
    
    param = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "auc")
    gbm = lgb.train(
        param, dtrain, valid_sets=[dvalid], verbose_eval=False, callbacks=[pruning_callback]
    )

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return accuracy

In [9]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=2134),
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10),
)
study.optimize(objective, n_trials=100, timeout=600)

[I 2022-12-20 19:57:23,675] A new study created in memory with name: no-name-5522f830-229f-432f-9bfb-b3bf07b19aee
c:\Users\4iste\anaconda3\envs\pygmo\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2022-12-20 19:57:23,796] Trial 0 finished with value: 0.7333333333333333 and parameters: {'bagging_fraction': 0.49985128604414425, 'bagging_freq': 3, 'min_child_samples': 21}. Best is trial 0 with value: 0.7333333333333333.
c:\Users\4iste\anaconda3\envs\pygmo\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecate

In [10]:
plot_optimization_history(study)

In [11]:
plot_intermediate_values(study)

In [12]:
plot_parallel_coordinate(study)

In [14]:
plot_contour(study)

In [16]:
plot_slice(study)

In [17]:
plot_param_importances(study)

In [18]:
plot_edf(study)